In [78]:
import numpy as np
import pandas as pd
import category_encoders
from sklearn.preprocessing import OneHotEncoder
from category_encoders import LeaveOneOutEncoder



def DataPreProcess(testSetValues, values_df, labels_df):

    df= pd.merge(values_df, labels_df, on = 'id' )

    for col in df.columns[df.isna().sum() > 0]:
        mode = df[col].mode()[0]
        df[col].fillna(value = mode, inplace = True)
        
    columnsToDrop = ['funder', 'num_private', 'subvillage', 'region_code', 'recorded_by', 'source_type', 'waterpoint_type', 'scheme_name', 'payment_type', 'quantity_group']
    df.drop(columns = columnsToDrop, inplace = True)
    df['status_group'] = df['status_group'].map({'functional': 2, 'functional needs repair': 1, 'non functional': 0})

    df['date_recorded'] = pd.to_datetime(df['date_recorded'])
    df['year_recorded'] = df['date_recorded'].dt.year
    df['month_recorded'] = df['date_recorded'].dt.month
    df.drop(columns = 'date_recorded', inplace = True)

    testSetValues = pd.read_csv('/home/DATA/TestSetValues.csv')

    for col in testSetValues.columns[testSetValues.isna().sum() > 0]:
        mode = testSetValues[col].mode()[0]
        testSetValues[col].fillna(value = mode, inplace = True)
         
    columnsToDrop = ['funder', 'num_private', 'subvillage', 'region_code', 'recorded_by', 'source_type', 'waterpoint_type', 'scheme_name', 'payment_type', 'quantity_group']
    testSetValues.drop(columns = columnsToDrop, inplace = True)

    testSetValues['date_recorded'] = pd.to_datetime(testSetValues['date_recorded'])
    testSetValues['year_recorded'] = testSetValues['date_recorded'].dt.year
    testSetValues['month_recorded'] = testSetValues['date_recorded'].dt.month
    testSetValues.drop(columns = 'date_recorded', inplace = True)


    target = 'status_group'

    encoder = LeaveOneOutEncoder()
    
    targetEncodeColumns = ['wpt_name', 'basin', 'region', 'district_code', 'lga', 'ward', 'scheme_management','installer','source',
                    'extraction_type', 'extraction_type_group', 'extraction_type_class','management', 'payment', 'water_quality', 'management_group', 'quality_group', 
        'quantity','source_class', 'waterpoint_type_group'] 

    for i in targetEncodeColumns:
        df[str(i) + '_encoded'] = encoder.fit_transform(df[i].values, df[target])
        testSetValues[str(i) + '_encoded'] = encoder.transform(testSetValues[i].values) 
        df.drop(columns=i, inplace=True) 
        testSetValues.drop(columns=i, inplace=True) 
    

    return df, testSetValues

In [36]:
testSetValues = pd.read_csv("/home/DATA/TestSetValues.csv")
trainSetLabels = pd.read_csv("/home/DATA/TrainingSetLabels.csv")
trainSetValues = pd.read_csv("/home/DATA/TrainingSetValues.csv")


In [79]:
df , value = DataPreProcess(testSetValues, trainSetLabels, trainSetValues)
df.head()

,id,status_group,amount_tsh,gps_height,longitude,latitude,population,public_meeting,permit,construction_year,year_recorded,month_recorded,wpt_name_encoded,basin_encoded,region_encoded,district_code_encoded,lga_encoded,ward_encoded,scheme_management_encoded,installer_encoded,source_encoded,extraction_type_encoded,extraction_type_group_encoded,extraction_type_class_encoded,management_encoded,payment_encoded,water_quality_encoded,management_group_encoded,quality_group_encoded,quantity_encoded,source_class_encoded,waterpoint_type_group_encoded
0,69572,2,6000.0,1390,34.938093,-9.856322,109,True,False,1999,2011,3,1.497473,1.356412,1.587568,5,1.568384,1.228571,1.087339,1.279570,1.319506,1.299339,1.299339,1.299339,1.077347,1.572370,1.208690,1.150870,1.208690,1.376947,1.144476,1.236945
1,8776,2,0.0,1399,34.698766,-2.147466,280,True,True,2010,2013,3,1.129071,1.091734,0.929878,2,1.220979,1.242857,1.223529,1.684211,1.344377,1.299339,1.299339,1.299339,1.297974,0.973015,1.208690,1.150870,1.208690,1.142253,1.208149,1.236945
2,34310,2,25.0,686,37.460664,-3.821329,250,True,True,2009,2013,2,2.000000,1.255062,1.307206,4,1.175896,1.777778,1.087339,1.439803,0.806107,1.299339,1.299339,1.299339,1.077347,1.401046,1.208690,1.150870,1.208690,1.376947,1.208149,1.236945
3,67743,0,0.0,263,38.486161,-11.155298,58,True,True,1986,2013,1,1.158838,0.816118,0.679005,63,0.535032,0.387097,1.087388,1.117647,1.023569,1.150325,1.118647,1.118647,1.077396,0.973093,1.208729,1.150908,1.208729,0.056205,1.144520,1.237003
4,19728,2,0.0,0,31.130847,-1.825359,0,True,True,0,2011,7,1.064110,1.091734,1.133032,1,1.118182,1.666667,1.087339,1.149254,1.344377,1.299339,1.299339,1.299339,1.260973,0.973015,1.208690,1.178344,1.208690,1.250679,1.208149,1.236945
